In [1]:
import re
import os
import glob
from urduhack import *
import subprocess
from flask import Flask, request, render_template,jsonify

In [9]:
stopwords = []

def get_all_headlines(folder_path):
    extraWordsinHeadlines = ['.doc','Urdu NEWS dataset//','voa','bbc','dataset','entertainment','sports','miscleneous','politics','//',"'"]
    headlines = []
    for root, dirs, files in os.walk(folder_path):
        for file in files:
            file_path = os.path.join(root, file)
            for word in extraWordsinHeadlines:
                file_path = file_path.replace(word,' ')
            headlines.append(file_path.strip())
    return headlines

def splitwords(headlines):
    lugat = []
    for headline in headlines:
        alfaz = re.split('\W+',headline)
        lugat.append(alfaz)
    return lugat


def readstopwords(file_name):
    global stopwords
    with open(file_name, "r",encoding='utf-8') as f:
        stopwords = [line.strip() for line in f.readlines()]
    return stopwords


def removeStopWords(words):
    #106033 -> 4571 after stopwords removal
    file_name = 'StopWordsUrduList.txt'
    temp = ''
    stopwords = readstopwords(file_name)
    #print(stopwords)
    wordswithoutstopwords = []
    for outer in words:
        for word in outer:
            if (word not in stopwords and word != '') and (word not in wordswithoutstopwords):
                temp += (' ' + word)
        temp = temp.strip()
        wordswithoutstopwords.append(temp)
        temp = ''
    return wordswithoutstopwords
                
    
def relatedNews(refinedDataset,Dataset,inputnews):
    relatedNewsList = []
    t = 0.3
    datasetlist = refinedDataset
    ni = inputnews
    sizeofdatasetlist = len(datasetlist)
    for j in range(0,sizeofdatasetlist):
        nj = datasetlist[j]
        Sij = getSimilarityScore(ni,nj)
        if Sij >= t:
            relatedNewsList.append(Dataset[j])
    #print(relatedNewsList)
    return relatedNewsList
    
    

def getTokensList(n):
    wordsWoSW = []
    words = []
    words = normalize(n)
    words = re.split('\W+',words)
    for word in words:
        if word in stopwords:
            continue
        wordsWoSW.append(word)
        
    return wordsWoSW

    
    
def getSimilarityScore(ni,nj):
    tli = getTokensList(ni)
    tlj = getTokensList(nj)
    sti = len(tli)
    stj = len(tlj)
    Sij = 0
    mij = 0
    
    for x in range(0,sti):
        for y in range(0,stj):
            if tli[x] == tlj[y]:
                mij = mij + 1
    
    if mij > 0:
        avg = (sti + stj)/2
        Sij = mij / avg
    
    return Sij



In [ ]:
app = Flask(__name__)
app.config['SEND_FILE_MAX_AGE_DEFAULT'] = 0
#app.debug = True
@app.route('/')
def index():
    #returning the index.html page on '/' request
    return render_template('/index.html')
    
@app.route('/evalexpression',methods=["POST"])
def evalexpression():
    #taking query from front-end
    data = request.get_json()
    data = data['expression']
    print(data)
    cluster = relatedNews(refinedHeadlines,Headlines,data)
    if(len(cluster) == 0):
        return jsonify({'message':"No Results Were Found!"})
    print("CLUSTER",cluster)
    response_data = {'message': cluster}
    return jsonify(response_data)



if __name__ == '__main__':

    file_path = 'refinedheadlines.txt'  # Replace with the path to your file

    with open(file_path, 'r',encoding='utf-8') as file:
        refinedHeadlines = file.readlines() 
    i = 0
    for line in refinedHeadlines:
        line = line.strip()
        refinedHeadlines[i] = line
        i += 1
    
    file_path = 'headlines.txt'  # Replace with the path to your file

    with open(file_path, 'r',encoding='utf-8') as file:
        Headlines = file.readlines()
    i = 0
    for line in Headlines:
        line = line.strip()
        Headlines[i] = line
        i += 1
    app.run(port=3000)

 * Serving Flask app '__main__'
 * Debug mode: off


 * Running on http://127.0.0.1:3000
Press CTRL+C to quit
127.0.0.1 - - [12/Jun/2023 00:02:00] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [12/Jun/2023 00:02:00] "GET /static/index.css HTTP/1.1" 304 -
127.0.0.1 - - [12/Jun/2023 00:02:00] "GET /static/script.js HTTP/1.1" 304 -
127.0.0.1 - - [12/Jun/2023 00:02:00] "GET /static/img/mainlogo.png HTTP/1.1" 304 -
127.0.0.1 - - [12/Jun/2023 00:02:00] "GET /static/img/bgimage.jpg HTTP/1.1" 304 -
127.0.0.1 - - [12/Jun/2023 00:02:00] "GET /static/img/searchBox.png HTTP/1.1" 304 -
127.0.0.1 - - [12/Jun/2023 00:02:00] "GET /static/img/buttons.png HTTP/1.1" 304 -
127.0.0.1 - - [12/Jun/2023 00:02:00] "GET /path/to/urdu-font.ttf HTTP/1.1" 404 -
127.0.0.1 - - [12/Jun/2023 00:02:04] "POST /evalexpression HTTP/1.1" 200 -


اقوام متحدہ میں شام میں جنگ بندی کے معاہدے کی توثیق
CLUSTER ['اقوام متحدہ میں شام میں جنگ بندی کے معاہدے کی توثیق', 'اقوام متحدہ کی قرارداد پر امریکہ اور اسرائیل کے درمیان سفارتی جنگ شروع ہو گئی ہے۔', 'روس شام میں 48 گھنٹوں کی جنگ بندی پر آمادہ', 'شام بین الاقوامی گروپ کا اجلاس، جنگ بندی برقرار رکھنے پر غور', 'شام جنگ بندی، امریکہ روس کسی سمجھوتے پر نہیں پہنچ پائے', 'شام میں جنگ بندی موثر، بعض مقامات پر جھڑپیں']


127.0.0.1 - - [12/Jun/2023 00:02:39] "POST /evalexpression HTTP/1.1" 200 -


افغان خاتون شربت گلہ کا اعتراف جرم، 15 دن کی سزا
CLUSTER ['افغان خاتون شربت گلہ کا اعتراف جرم، 15 دن کی سزا', 'افغان خاتون شربت گلہ کو 15 دن کی سزا', 'شربت گلہ کو 15 دن قید اور جرمانے کی سزا']


...

افغانستان ہم جنس پرستوں کی خفیہ زندگی
CLUSTER ['افغانستان ہم جنس پرستوں کی خفیہ زندگی', '’ہم جنس پرستوں کے حقوق کے تحفظ کا وعدہ‘']


127.0.0.1 - - [12/Jun/2023 00:05:05] "POST /evalexpression HTTP/1.1" 200 -


ویسٹ انڈیز اور پاکستان دوسرے ون ڈے میچ میں مدِمقابل
CLUSTER ['آسٹریلیا کے خلاف پاکستان کی ون ڈے ٹیم کا اعلان، عرفان کی واپسی', 'تیسرا ون ڈے پاکستان اور ویسٹ انڈیز ابوظہبی میں مدِمقابل ہوں گی', 'شارجہ میں ویسٹ انڈیز اور پاکستان پہلے ون ڈے میچ میں مدِمقابل', 'ون ڈے سیریز پاکستان کے لیے رینکنگ بہتر بنانے کا موقع', 'ون ڈے میں بھی کلین سویپ، پاکستان عالمی درجہ بندی میں آٹھویں پوزیشن پر', 'ویسٹ انڈیز اور پاکستان دوسرے ون ڈے میچ میں مدِمقابل', 'پاکستان دوسرے ون ڈے میں بھی کامیاب', 'پاکستان ون ڈے سیریز کے آخری میچ کا فاتح', 'پاکستان ویسٹ انڈیز کے خلاف ٹیسٹ سیریز میں فتح تصاویر', 'پاکستان کی ویسٹ انڈیز کے خلاف111 رنز سے جیت', 'دبئی ٹیسٹ ویسٹ\xa0انڈیز کو میچ جیتنے کے لئے251 رن درکار', 'دوسرا ٹیسٹ ویسٹ انڈیز کے خلاف پاکستان کے 304 رنز، چار آؤٹ', 'ویسٹ انڈیز ہدف حاصل کرنے میں ناکام، پاکستان سیریز جیت گیا', 'پاکستان بمقابلہ ویسٹ انڈیز ، پہلا ٹیسٹ جمعرات کو', 'پاکستان بمقابلہ ویسٹ انڈیز،دوسرا ٹیسٹ جمعہ سے', 'پاکستان نے ون ڈے سیزیز جیت لی، ویسٹ انڈیز کو شکست', 'پاکستان کی ویسٹ انڈیز کو پھر ’کلین سوئپ

127.0.0.1 - - [12/Jun/2023 00:05:41] "POST /evalexpression HTTP/1.1" 200 -


انڈیا کے زیرِ انتظام کشمیر میں سکیورٹی ہٹنے سے پنڈت خوفزدہ
CLUSTER ['انڈیا کے زیر انتظام کشمیر کی نو سالہ لڑکی کی کک باکسنگ پر حکمرانی', 'انڈیا کے زیرِ انتظام کشمیر میں سکیورٹی ہٹنے سے پنڈت خوفزدہ', 'انڈیا کے زیرِ انتظام کشمیر میں سیکورٹی ہٹانے سے پنڈت آبادی خوفزدہ']


In [53]:
def writeToFile(refinedheadlines,headlines):
    #print(headlines)
    with open('refinedheadlines.txt', 'w',encoding='utf-8') as file:
        for hdline in refinedheadlines:
            file.write(hdline + '\n')
    with open('headlines.txt', 'w',encoding='utf-8') as file:
        for hdline in headlines:
            file.write(hdline + '\n')
            
            
folder_path = 'Urdu NEWS dataset\\'
headlines = get_all_headlines(folder_path)
words = splitwords(headlines)
refinedHeadlines = removeStopWords(words)
writeToFile(refinedHeadlines,headlines)

['زیادہ پانی پینا بھی خطرناک ہو سکتا ہے', 'شاہ رخ سے رومانس نہ کرنے کا ملال نہیں', 'مومنہ ایک سیلفی پلیز', 'پرینکا مس ورلڈ بنیں تو میں سکول میں تھی', 'گیلیکسی نوٹ سیون کی پروڈکشن بند', '100 خواتین ایزابیلا کے لیے امتیازی سلوک ایک مہمیز ثابت ہوا', '12 گھنٹوں میں مریخ پر رہنا سیکھیں', '14 نومبر کو معمول سے زیادہ بڑا اور روشن چاند', '17 کروڑ ڈالر کے قیمتی پتھر ’یشمِ سبز‘ کی دریافت', '20 سال کے بعد اروندھتی رائے کے نئے ناول کی اشاعت کا اعلان', '2016 میں پاکستانی ڈرامہ نڈر ہو گیا؟', '2016 کی اہم دریافتیں دوست بلیاں اور ہینی بال کے گھوڑے', '2016 کے دوران طب کی دنیا میں برفانی پانی، زیکا اور تین والدین کا بچہ', '2016 کے مقبول ٹوئٹر ٹرینڈز', '2025 سے ڈیزل سے چلنے والی گاڑیوں پر پابندی کا اعلان', '86 بیویوں والا شخص چل بسا', 'آئی فون کے دس سال سمارٹ فون اتنا سمارٹ کیسے بنا؟', 'آئی فونز کی فروخت توقع سے زیادہ تاہم منافع میں کمی', 'آخری رسومات عقائد کے برخلاف', 'آسٹریلیا کپڑوں میں لپٹی ڈولفن کی تلاش', 'آسکر ایوارڈز میں ملبوسات و جواہرات کی چمک', 'آسکرز ایوارڈ 2017 تصاویر میں', 'آلودگی کے ذرّات کی